In [4]:
import torch 
import yaml, os
import matplotlib, seaborn as sns
from torch_geometric.data import HeteroData, Dataset, Data as PyGData
from pytorch_lightning import LightningModule
from itertools import combinations_with_replacement, product
from typing import Dict, Optional
from torch import Tensor
from torch_geometric.typing import Adj, EdgeType, NodeType
import pdb
import numpy as np
import sys 
sys.path.append('/global/cfs/cdirs/m3443/usr/pmtuan/Tracking-ML-Exa.TrkX')
%matplotlib inline
from itertools import product

CONFIGFILE = "hetero_reg3_lev3.yaml"
with open(CONFIGFILE, 'r') as f: config = yaml.load(f, Loader=yaml.FullLoader)
data_dir = config['input_dir'] + '/train'
data = torch.load(os.path.join(data_dir, os.listdir(data_dir)[0]))

In [5]:
from Pipelines.Common_Tracking_Example.LightningModules.GNN.Models.hetero_gnn import PyGHeteroGNN
gnn = PyGHeteroGNN(config)
gnn.setup()

Setting up dataset
using in-fly dataset


In [65]:
het_data = gnn.trainset.get(0)
from utils import load_dataset, process_data
homo_data = process_data(torch.load(gnn.trainset.input_paths[0]), 100, 1000, True, False, None)[0]

1it [00:00, 95.99it/s]
1it [00:00, 15.98it/s]


In [19]:
homo_data , het_data.to_homogeneous()

(Data(x=[301736, 9], cell_data=[301736, 11], pid=[301736], event_file='/global/cfs/cdirs/m3443/data/ITk-upgrade/processed/full_events_v4/event000014435', hid=[301736], pt=[301736], primary=[301736], nhits=[301736], modules=[301736], modulewise_true_edges=[2, 119372], signal_true_edges=[2, 13354], edge_index=[2, 421007], y=[421007], y_pid=[421007], volume_id=[301736], pid_signal=[421007]),
 Data(event_file='/global/cfs/cdirs/m3443/data/ITk-upgrade/processed/full_events_v4/event000014435', modulewise_true_edges=[2, 119372], signal_true_edges=[2, 13354], edge_index=[2, 421007], cell_data=[301736, 11], pid=[301736], hid=[301736], pt=[301736], primary=[301736], nhits=[301736], modules=[301736], volume_id=[301736], y=[421007], y_pid=[421007], pid_signal=[421007], node_type=[301736], edge_type=[421007]))

In [21]:
(homo_data.hid == het_data.to_homogeneous().hid).sum()

tensor(246754)

In [25]:
(homo_data.hid - het_data.to_homogeneous().hid).nonzero()

tensor([[229591],
        [229592],
        [229593],
        ...,
        [284570],
        [284571],
        [284572]])

In [39]:
(homo_data.nhits - het_data.to_homogeneous().nhits).nonzero()

tensor([[229591],
        [229593],
        [229594],
        ...,
        [284569],
        [284571],
        [284572]])

In [35]:
nz = (het_data.to_homogeneous().volume_id - homo_data.volume_id).nonzero()
nz

tensor([[229591],
        [229592],
        [229593],
        ...,
        [284570],
        [284571],
        [284572]])

In [48]:
(~(homo_data.edge_index.T == het_data.to_homogeneous().edge_index.T
)).all(1).float().nonzero().size(0)

348739

In [66]:
for key in het_data.edge_index_dict:
    het_data[key]['my_var'] = het_data.y_dict[key]

In [67]:
het_data

HeteroData(
  event_file='/global/cfs/cdirs/m3443/data/ITk-upgrade/processed/full_events_v4/event000014435',
  modulewise_true_edges=[2, 119372],
  signal_true_edges=[2, 13354],
  volume_0_1={
    x=[229591, 3],
    cell_data=[229591, 11],
    pid=[229591],
    hid=[229591],
    pt=[229591],
    primary=[229591],
    nhits=[229591],
    modules=[229591],
    volume_id=[229591]
  },
  volume_2={
    x=[38135, 9],
    cell_data=[38135, 11],
    pid=[38135],
    hid=[38135],
    pt=[38135],
    primary=[38135],
    nhits=[38135],
    modules=[38135],
    volume_id=[38135]
  },
  volume_3={
    x=[34010, 9],
    cell_data=[34010, 11],
    pid=[34010],
    hid=[34010],
    pt=[34010],
    primary=[34010],
    nhits=[34010],
    modules=[34010],
    volume_id=[34010]
  },
  (volume_0_1, connected_to, volume_0_1)={
    edge_index=[2, 263044],
    y=[263044],
    y_pid=[263044],
    pid_signal=[263044],
    my_var=[263044]
  },
  (volume_0_1, connected_to, volume_2)={
    edge_index=[2, 20188]

In [68]:
het_data.to_homogeneous()

Data(event_file='/global/cfs/cdirs/m3443/data/ITk-upgrade/processed/full_events_v4/event000014435', modulewise_true_edges=[2, 119372], signal_true_edges=[2, 13354], edge_index=[2, 421007], cell_data=[301736, 11], pid=[301736], hid=[301736], pt=[301736], primary=[301736], nhits=[301736], modules=[301736], volume_id=[301736], y=[421007], y_pid=[421007], pid_signal=[421007], my_var=[421007], node_type=[301736], edge_type=[421007])

In [52]:
het_data.to_homogeneous()

Data(
  event_file='/global/cfs/cdirs/m3443/data/ITk-upgrade/processed/full_events_v4/event000014435',
  modulewise_true_edges=[2, 119372],
  signal_true_edges=[2, 13354],
  edge_newindex_dict={
    (volume_0_1, connected_to, volume_0_1)=[2, 263044],
    (volume_0_1, connected_to, volume_2)=[2, 20188],
    (volume_0_1, connected_to, volume_3)=[2, 8641],
    (volume_2, connected_to, volume_0_1)=[2, 21954],
    (volume_2, connected_to, volume_2)=[2, 61249],
    (volume_2, connected_to, volume_3)=[2, 6685],
    (volume_3, connected_to, volume_0_1)=[2, 9441],
    (volume_3, connected_to, volume_2)=[2, 6343],
    (volume_3, connected_to, volume_3)=[2, 23462]
  },
  edge_index=[2, 421007],
  cell_data=[301736, 11],
  pid=[301736],
  hid=[301736],
  pt=[301736],
  primary=[301736],
  nhits=[301736],
  modules=[301736],
  volume_id=[301736],
  y=[421007],
  y_pid=[421007],
  pid_signal=[421007],
  node_type=[301736],
  edge_type=[421007]
)

In [34]:
het_data.to_homogeneous().volume_id[nz]

tensor([[2],
        [2],
        [2],
        ...,
        [3],
        [3],
        [3]])

In [32]:
homo_data.volume_id[nz]

tensor([[3],
        [3],
        [3],
        ...,
        [2],
        [2],
        [2]])

In [22]:
gnn.trainset[0].collect('pid_signal'), gnn.trainset[0].y_pid_dict

({('volume_0_1',
   'connected_to',
   'volume_0_1'): tensor([False, False, False,  ..., False, False, False]),
  ('volume_0_1',
   'connected_to',
   'volume_2'): tensor([False, False, False,  ..., False, False, False]),
  ('volume_0_1',
   'connected_to',
   'volume_3'): tensor([False, False, False,  ..., False, False, False]),
  ('volume_2',
   'connected_to',
   'volume_0_1'): tensor([False, False, False,  ..., False, False, False]),
  ('volume_2',
   'connected_to',
   'volume_2'): tensor([False, False, False,  ..., False,  True, False]),
  ('volume_2',
   'connected_to',
   'volume_3'): tensor([False, False, False,  ..., False, False, False]),
  ('volume_3',
   'connected_to',
   'volume_0_1'): tensor([False, False, False,  ..., False, False, False]),
  ('volume_3',
   'connected_to',
   'volume_2'): tensor([False, False, False,  ..., False, False, False]),
  ('volume_3',
   'connected_to',
   'volume_3'): tensor([False, False, False,  ..., False, False, False])},
 {('volume_0_1'

In [3]:
from pytorch_lightning import Trainer

trainer = Trainer(max_epochs=100, accelerator='gpu', devices=1, detect_anomaly=True)

trainer.fit(gnn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Setting up dataset
Loading events from /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/hetero_processed/0GeV_v3/train


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:28<00:00,  3.51it/s]


Events loaded!


100it [00:02, 39.51it/s]
100it [00:05, 17.13it/s]


...

Epoch 87:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:25<00:20,  2.05s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 88:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:26<00:20,  2.07s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 89:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:27<00:20,  2.07s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 89:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍       | 104/110 [03:28<00:12,  2.01s/it, loss=0.55, v_num=3549793]


Epoch 90:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:29<00:20,  2.09s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 91:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:52<00:23,  2.33s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 92:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:37<00:21,  2.17s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:  70%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                             | 7/10 [00:01<00:00,  4.75it/s]


Epoch 93:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:28<00:20,  2.08s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 94:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:28<00:20,  2.09s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 95:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:27<00:20,  2.07s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 95:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 109/110 [03:30<00:01,  1.93s/it, loss=0.55, v_num=3549793]


Epoch 96:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:30<00:21,  2.10s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 97:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:29<00:20,  2.09s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 98:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:33<00:21,  2.13s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Epoch 99:  91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 100/110 [03:26<00:20,  2.06s/it, loss=0.55, v_num=3549793]
Validation: 0it [00:00, ?it/s]
Validation DataLoade

Epoch 99: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [03:29<00:00,  1.91s/it, loss=0.55, v_num=3549793]


In [6]:
from Pipelines.Common_Tracking_Example.LightningModules.GNN.utils import make_mlp, get_region

class NodeEncoder(torch.nn.Module):
    def __init__(self, hparams, model) -> None:
        super().__init__()
        self.hparams = hparams

        self.network = make_mlp(
            model['num_features'],
            [hparams["hidden"]] * hparams["nb_node_layer"],
            output_activation=hparams["output_activation"],
            hidden_activation=hparams["hidden_activation"],
            layer_norm=hparams["layernorm"],
            batch_norm=hparams["batchnorm"],
        )

    def forward(self, x: torch.Tensor):
        return  self.network( x.float() )

class HeteroNodeEncoder(torch.nn.Module):
    def __init__(self, hparams) -> None:
        super().__init__()

        self.hparams = hparams

        self.encoders = torch.nn.ModuleDict()
        for model in self.hparams['model_ids']:
            region = get_region(model)
            self.encoders[region] = NodeEncoder(self.hparams, model)
        
    def forward(self, x_dict):
        for region, enc in self.encoders.items():
            x_dict[region] = enc(x_dict[region])
        return x_dict

class EdgeEncoder(torch.nn.Module):
    def __init__(self, hparams) -> None:
        super().__init__()
        self.hparams = hparams

        self.network = make_mlp(
            2 * hparams['hidden'],
            [hparams["hidden"]] * hparams["nb_node_layer"],
            output_activation=hparams["output_activation"],
            hidden_activation=hparams["hidden_activation"],
            layer_norm=hparams["layernorm"],e, 
            batch_norm=hparams["batchnorm"],
        )

    def forward(self, x, edge_index):
        src, dst = edge_index
        if isinstance(x, tuple):
            x1, x2 = x
            x_in = torch.cat([x1[src], x2[dst]], dim=-1)
        else:
            x_in = torch.cat([x[src], x[dst]], dim=-1)

        return  self.network( x_in )
    
class EdgeClassifier(torch.nn.Module):

    def __init__(self, hparams):
        super().__init__()

        self.network = make_mlp(
            3 * hparams["hidden"],
            [hparams["hidden"]] * hparams["nb_edge_layer"] + [1],
            layer_norm=hparams["layernorm"],
            batch_norm=hparams["batchnorm"],
            output_activation=hparams['output_activation'],
            hidden_activation=hparams["hidden_activation"],
        )

    def forward(self, x, edge_index, edge):
        src, dst = edge_index
        if isinstance(x, tuple):
            x1, x2 = x
            classifier_input = torch.cat([x1[src], x2[dst], edge], dim=-1)
        else:
            classifier_input = torch.cat([x[src], x[dst], edge], dim=-1)
        return self.network(classifier_input).squeeze()

from torch_geometric.nn import HeteroConv
    
class HeteroEdgeConv(HeteroConv):

    def __init__(self, convs: dict, aggr: str = "sum"):
        super().__init__(convs, aggr)

    def forward(
        self,
        x_dict: dict,
        edge_index_dict: dict,
        *args_dict,
        **kwargs_dict,
    ) -> dict :

        out_dict = {}
        for edge_type, edge_index in edge_index_dict.items():
            src, rel, dst = edge_type
            str_edge_type = '__'.join(edge_type)
            if str_edge_type not in self.convs:
                continue

            args = []
            for value_dict in args_dict:
                if edge_type in value_dict:
                    args.append(value_dict[edge_type])
                elif src == dst and src in value_dict:
                    args.append(value_dict[src])
                elif src in value_dict or dst in value_dict:
                    args.append(
                        (value_dict.get(src, None), value_dict.get(dst, None)))

            kwargs = {}
            for arg, value_dict in kwargs_dict.items():
                arg = arg[:-5]  # `{*}_dict`
                if edge_type in value_dict:
                    kwargs[arg] = value_dict[edge_type]
                elif src == dst and src in value_dict:
                    kwargs[arg] = value_dict[src]
                elif src in value_dict or dst in value_dict:
                    kwargs[arg] = (value_dict.get(src, None),
                                value_dict.get(dst, None))

            conv = self.convs[str_edge_type]

            if src == dst:
                out = conv(x_dict[src], edge_index, *args, **kwargs)
            else:
                out = conv((x_dict[src], x_dict[dst]), edge_index, *args, **kwargs)

            out_dict[edge_type] = out

        return out_dict

module = HeteroNodeEncoder(config)

edge_module = HeteroEdgeConv({
        (get_region(model0), 'connected_to', get_region(model1)): EdgeEncoder(config)
        for model0, model1 in product(config['model_ids'], config['model_ids'])
    })

edge_classifier = HeteroEdgeConv({
        (get_region(model0), 'connected_to', get_region(model1)): EdgeClassifier(config)
        for model0, model1 in product(config['model_ids'], config['model_ids'])
    })

x_dict = module(batch.x_dict)

edge_dict = edge_module(x_dict, batch.edge_index_dict)

edge_out = edge_classifier(x_dict, batch.edge_index_dict, edge_dict = edge_dict)

truth_dict = batch.truth_dict

loss = torch.nn.functional.binary_cross_entropy_with_logits(
    torch.cat(list(edge_out.values()), dim=0), 
    torch.cat(list(truth_dict.values()), dim=0).float() ,reduction='mean'#
)

loss.backward()
for param in list(module.encoders.values())[0].parameters():
    print(param.grad)


SyntaxError: positional argument follows keyword argument (2148637783.py, line 46)

In [3]:
gnn.setup()
batch=gnn.train_dataloader().dataset[1]
# loss = gnn.training_step(batch, 0, log=False)
# loss
gnn(batch.x_dict, batch.edge_index_dict)

Setting up dataset
Loading events from /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/hetero_processed/0GeV_v3/train


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.91it/s]


Events loaded!


5it [00:00, 96.27it/s]
5it [00:00, 11.15it/s]


Events processed!
Loading events from /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/hetero_processed/0GeV_v3/val


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.88it/s]


Events loaded!


1it [00:00, 154.64it/s]
1it [00:00, 13.12it/s]


Events processed!


{'volume_0_1': tensor([[-0.7785],
         [-0.7771],
         [-0.7730],
         ...,
         [ 0.0545],
         [ 0.0739],
         [-0.0145]], grad_fn=<TanhBackward0>),
 'volume_2': tensor([[-0.9568],
         [-0.9731],
         [-0.9731],
         ...,
         [-0.4402],
         [-0.4410],
         [-0.2864]], grad_fn=<TanhBackward0>),
 'volume_3': tensor([[ 0.2491],
         [ 0.2330],
         [ 0.2498],
         ...,
         [-0.1688],
         [-0.2285],
         [-0.2325]], grad_fn=<TanhBackward0>)}

In [4]:
batch

HeteroData(
  volume_0_1={
    x=[231415, 3],
    cell_data=[231415, 11],
    pid=[231415],
    hid=[231415],
    pt=[231415],
    primary=[231415],
    nhits=[231415],
    modules=[231415],
    volume_id=[231415],
    mask=[297886]
  },
  volume_2={
    x=[33091, 9],
    cell_data=[33091, 11],
    pid=[33091],
    hid=[33091],
    pt=[33091],
    primary=[33091],
    nhits=[33091],
    modules=[33091],
    volume_id=[33091],
    mask=[297886]
  },
  volume_3={
    x=[33380, 9],
    cell_data=[33380, 11],
    pid=[33380],
    hid=[33380],
    pt=[33380],
    primary=[33380],
    nhits=[33380],
    modules=[33380],
    volume_id=[33380],
    mask=[297886]
  },
  (volume_0_1, connected_to, volume_0_1)={
    edge_index=[2, 252287],
    y=[252287],
    y_pid=[252287],
    truth=[252287],
    modulewise_true_edges=[2, 98741],
    signal_true_edges=[2, 10198]
  },
  (volume_0_1, connected_to, volume_2)={
    edge_index=[2, 16525],
    y=[16525],
    y_pid=[16525],
    truth=[16525],
    modu

In [5]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear


dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('paper', 'cites', 'paper'): GCNConv(-1, hidden_channels),
                ('author', 'writes', 'paper'): SAGEConv((-1, -1), hidden_channels),
                ('paper', 'rev_writes', 'author'): GATConv((-1, -1), hidden_channels, add_self_loops=False),
            }, aggr='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        return self.lin(x_dict['author'])

model = HeteroGNN(hidden_channels=64, out_channels=dataset.num_classes,
                  num_layers=2)

In [6]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)

In [7]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader
import torch.nn.functional as F

transform = T.ToUndirected()  # Add reverse edge types.
data = OGB_MAG(root='./data', preprocess='metapath2vec', transform=transform)[0]

train_loader = NeighborLoader(
    data,
    # Sample 15 neighbors for each node and each edge type for 2 iterations:
    num_neighbors=[15] * 2,
    # Use a batch size of 128 for sampling training nodes of type "paper":
    batch_size=128,
    input_nodes=('paper', data['paper'].train_mask),
)

device = 'cuda:0'
model.to(device)
model.train()

total_examples = total_loss = 0

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
print_grad = True
# for batch in train_loader:
#     optimizer.zero_grad()
#     batch = batch.to('cuda:0')
#     batch_size = batch['paper'].batch_size
#     out = model(batch.x_dict, batch.edge_index_dict)
#     loss = F.cross_entropy(out[:batch_size],
#                            batch['paper'].y[:batch_size])
#     print(loss)
#     loss.backward()
#     if print_grad:
#         for param in model.parameters():
#             print(param.grad)
#     print_grad=False
#     optimizer.step()

#     total_examples += batch_size
#     total_loss += float(loss) * batch_size

In [12]:
for batch in train_loader:
    print(batch)
    

HeteroData(
  paper={
    x=[20853, 128],
    year=[20853],
    y=[20853],
    train_mask=[20853],
    val_mask=[20853],
    test_mask=[20853],
    batch_size=128
  },
  author={ x=[4384, 128] },
  institution={ x=[314, 128] },
  field_of_study={ x=[2604, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5850] },
  (paper, cites, paper)={ edge_index=[2, 11797] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 10573] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 838] },
  (paper, rev_writes, author)={ edge_index=[2, 5481] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 10461] }
)
HeteroData(
  paper={
    x=[22179, 128],
    year=[22179],
    y=[22179],
    train_mask=[22179],
    val_mask=[22179],
    test_mask=[22179],
    batch_size=128
  },
  author={ x=[4283, 128] },
  institution={ x=[326, 128] },
  field_of_study={ x=[2755, 128] },
  (author, affiliated_with, institution)={ edg

HeteroData(
  paper={
    x=[20625, 128],
    year=[20625],
    y=[20625],
    train_mask=[20625],
    val_mask=[20625],
    test_mask=[20625],
    batch_size=128
  },
  author={ x=[3601, 128] },
  institution={ x=[271, 128] },
  field_of_study={ x=[2798, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 4894] },
  (paper, cites, paper)={ edge_index=[2, 11191] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 11299] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 674] },
  (paper, rev_writes, author)={ edge_index=[2, 4474] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 10483] }
)
HeteroData(
  paper={
    x=[22836, 128],
    year=[22836],
    y=[22836],
    train_mask=[22836],
    val_mask=[22836],
    test_mask=[22836],
    batch_size=128
  },
  author={ x=[4430, 128] },
  institution={ x=[317, 128] },
  field_of_study={ x=[2946, 128] },
  (author, affiliated_with, institution)={ edg

HeteroData(
  paper={
    x=[23112, 128],
    year=[23112],
    y=[23112],
    train_mask=[23112],
    val_mask=[23112],
    test_mask=[23112],
    batch_size=128
  },
  author={ x=[4454, 128] },
  institution={ x=[334, 128] },
  field_of_study={ x=[3058, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5774] },
  (paper, cites, paper)={ edge_index=[2, 13492] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 11461] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 789] },
  (paper, rev_writes, author)={ edge_index=[2, 4657] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 11770] }
)
HeteroData(
  paper={
    x=[23288, 128],
    year=[23288],
    y=[23288],
    train_mask=[23288],
    val_mask=[23288],
    test_mask=[23288],
    batch_size=128
  },
  author={ x=[4754, 128] },
  institution={ x=[342, 128] },
  field_of_study={ x=[2925, 128] },
  (author, affiliated_with, institution)={ edg

HeteroData(
  paper={
    x=[22503, 128],
    year=[22503],
    y=[22503],
    train_mask=[22503],
    val_mask=[22503],
    test_mask=[22503],
    batch_size=128
  },
  author={ x=[4630, 128] },
  institution={ x=[282, 128] },
  field_of_study={ x=[2970, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5746] },
  (paper, cites, paper)={ edge_index=[2, 12186] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 11616] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 756] },
  (paper, rev_writes, author)={ edge_index=[2, 4905] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 11294] }
)
HeteroData(
  paper={
    x=[23137, 128],
    year=[23137],
    y=[23137],
    train_mask=[23137],
    val_mask=[23137],
    test_mask=[23137],
    batch_size=128
  },
  author={ x=[4354, 128] },
  institution={ x=[302, 128] },
  field_of_study={ x=[3295, 128] },
  (author, affiliated_with, institution)={ edg

HeteroData(
  paper={
    x=[22252, 128],
    year=[22252],
    y=[22252],
    train_mask=[22252],
    val_mask=[22252],
    test_mask=[22252],
    batch_size=128
  },
  author={ x=[3985, 128] },
  institution={ x=[351, 128] },
  field_of_study={ x=[3012, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5005] },
  (paper, cites, paper)={ edge_index=[2, 12194] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 11557] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 803] },
  (paper, rev_writes, author)={ edge_index=[2, 4390] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 11200] }
)
HeteroData(
  paper={
    x=[20822, 128],
    year=[20822],
    y=[20822],
    train_mask=[20822],
    val_mask=[20822],
    test_mask=[20822],
    batch_size=128
  },
  author={ x=[3800, 128] },
  institution={ x=[249, 128] },
  field_of_study={ x=[2958, 128] },
  (author, affiliated_with, institution)={ edg

...

HeteroData(
  paper={
    x=[24551, 128],
    year=[24551],
    y=[24551],
    train_mask=[24551],
    val_mask=[24551],
    test_mask=[24551],
    batch_size=128
  },
  author={ x=[4512, 128] },
  institution={ x=[340, 128] },
  field_of_study={ x=[3145, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5970] },
  (paper, cites, paper)={ edge_index=[2, 14129] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 12286] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 886] },
  (paper, rev_writes, author)={ edge_index=[2, 5499] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 12430] }
)
HeteroData(
  paper={
    x=[25138, 128],
    year=[25138],
    y=[25138],
    train_mask=[25138],
    val_mask=[25138],
    test_mask=[25138],
    batch_size=128
  },
  author={ x=[5161, 128] },
  institution={ x=[365, 128] },
  field_of_study={ x=[3156, 128] },
  (author, affiliated_with, institution)={ edg

HeteroData(
  paper={
    x=[22823, 128],
    year=[22823],
    y=[22823],
    train_mask=[22823],
    val_mask=[22823],
    test_mask=[22823],
    batch_size=128
  },
  author={ x=[4328, 128] },
  institution={ x=[320, 128] },
  field_of_study={ x=[2993, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5580] },
  (paper, cites, paper)={ edge_index=[2, 12959] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 11288] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 770] },
  (paper, rev_writes, author)={ edge_index=[2, 4510] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 12076] }
)
HeteroData(
  paper={
    x=[22708, 128],
    year=[22708],
    y=[22708],
    train_mask=[22708],
    val_mask=[22708],
    test_mask=[22708],
    batch_size=128
  },
  author={ x=[3991, 128] },
  institution={ x=[318, 128] },
  field_of_study={ x=[2753, 128] },
  (author, affiliated_with, institution)={ edg

HeteroData(
  paper={
    x=[23256, 128],
    year=[23256],
    y=[23256],
    train_mask=[23256],
    val_mask=[23256],
    test_mask=[23256],
    batch_size=128
  },
  author={ x=[4222, 128] },
  institution={ x=[317, 128] },
  field_of_study={ x=[2951, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5534] },
  (paper, cites, paper)={ edge_index=[2, 12446] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 12064] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 771] },
  (paper, rev_writes, author)={ edge_index=[2, 4995] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 11395] }
)
HeteroData(
  paper={
    x=[22597, 128],
    year=[22597],
    y=[22597],
    train_mask=[22597],
    val_mask=[22597],
    test_mask=[22597],
    batch_size=128
  },
  author={ x=[4022, 128] },
  institution={ x=[331, 128] },
  field_of_study={ x=[3092, 128] },
  (author, affiliated_with, institution)={ edg

HeteroData(
  paper={
    x=[23055, 128],
    year=[23055],
    y=[23055],
    train_mask=[23055],
    val_mask=[23055],
    test_mask=[23055],
    batch_size=128
  },
  author={ x=[4170, 128] },
  institution={ x=[331, 128] },
  field_of_study={ x=[2866, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5333] },
  (paper, cites, paper)={ edge_index=[2, 12243] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 11871] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 740] },
  (paper, rev_writes, author)={ edge_index=[2, 4440] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 11419] }
)


KeyboardInterrupt: 

In [3]:
from functools import partial

def get_region(model):
    return 'volume_' + '_'.join([str(i) for i in model['volume_ids']])

def process_data(events, pt_background_cut, pt_signal_cut, noise, triplets, input_cut):
    # Handle event in batched form
    if type(events) is not list:
        events = [events]

    # NOTE: Cutting background by pT BY DEFINITION removes noise
    if pt_background_cut > 0 or not noise:
        for i, event in enumerate(events):

            if triplets:  # Keep all event data for posterity!
                event = convert_triplet_graph(event)

            else:
                event = background_cut_event(event, pt_background_cut, pt_signal_cut)                
                    
    for i, event in enumerate(events):
        
        # Ensure PID definition is correct
        event.y_pid = (event.pid[event.edge_index[0]] == event.pid[event.edge_index[1]]) & event.pid[event.edge_index[0]].bool()
        event.pid_signal = torch.isin(event.edge_index, event.signal_true_edges).all(0) & event.y_pid
        
        if (input_cut is not None) and "scores" in event.keys:
            score_mask = event.scores > input_cut
            for edge_attr in ["edge_index", "y", "y_pid", "pid_signal", "scores"]:
                event[edge_attr] = event[edge_attr][..., score_mask]            

    return events[0]

# class LargeHeteroDataset(Dataset):
#     def __init__(self, root, subdir, hparams, num_events=-1, transform=None, pre_transform=None, pre_filter=None):
#         super().__init__(root, transform, pre_transform, pre_filter)
        
#         self.subdir = subdir
#         self.hparams = hparams
#         # if transform is not None:
#         #     from functools import partial
#         #     self.transform = partial(transform, pt_background_cut=self.hparams['pt_background_cut'], pt_signal_cut=self.hparams['pt_signal_cut'], noise=self.hparams['noise'], triplets=False, input_cut=None)
        
#         self.input_paths = os.listdir(os.path.join(root, subdir))
#         if "sorted_events" in hparams.keys() and hparams["sorted_events"]:
#             self.input_paths = sorted(self.input_paths)
#         else:
#             random.shuffle(self.input_paths)
        
#         self.input_paths = [os.path.join(root, subdir, event) for event in self.input_paths][:num_events]
        
#     def len(self):
#         return len(self.input_paths)
    
#     def get(self, idx):
#         event = torch.load(self.input_paths[idx], map_location=torch.device("cpu"))     

#         map = torch.zeros_like(event.hid)
#         for model in self.hparams['model_ids']:
#             volume_id = model['volume_ids']
#             homo_ids = event.hid[ torch.isin( event.volume_id, torch.tensor(volume_id) ) ]
#             map[homo_ids] = torch.arange(homo_ids.shape[0])
        

#         data = HeteroData()
#         for model in self.hparams['model_ids']:
#             region = get_region(model)
#             mask = torch.isin( event.volume_id, torch.tensor(model['volume_ids']) )
#             for attr in ['x', 'cell_data', 'pid', 'hid', 'pt', 'primary', 'nhits', 'modules', 'volume_id']:
#                 data[region][attr] = event[attr][mask]
#             data[region]['mask'] = mask
        
#         for model1, model2 in product(self.hparams['model_ids'], self.hparams['model_ids']):
#             # ids = torch.tensor([model1['volume_ids'], model2['volume_ids']])
#             id0, id1 = torch.tensor([model1['volume_ids']]), torch.tensor([model2['volume_ids']])
#             region1, region2 = get_region(model1), get_region(model2)
#             mask0 = torch.isin(event.volume_id[event.edge_index[0]], id0)
#             mask1 = torch.isin(event.volume_id[event.edge_index[1]], id1)
#             mask = mask1 * mask0 #+ torch.isin(event.volume_id[event.edge_index[0]], id2) * torch.isin(event.volume_id[event.edge_index[1]],id1)
#             edge_index = event.edge_index.T[mask].T
#             edge_index = map[edge_index]
#             data[region1, 'connected_to', region2].edge_index = edge_index
#             data[region1, 'connected_to', region2].y = event.y[mask]
#             data[region1, 'connected_to', region2].y_pid = event.y_pid[mask]
#             for truth_edge in ['modulewise_true_edges', 'signal_true_edges']:
#                 mask = torch.isin(event.volume_id[event[truth_edge][0]], id0) * torch.isin(event.volume_id[event[truth_edge][1]], id1) #+ torch.isin(event.volume_id[event[truth_edge][0]], id2) * torch.isin(event.volume_id[event[truth_edge][1]], id1)
#                 data[region1, 'connected_to', region2][truth_edge] = event[truth_edge].T[mask].T
#         return data 

# class LargeHeteroDataset(LargeDataset):
#     def __init__(self, root, subdir, hparams, num_events=-1, transform=None, pre_transform=None, pre_filter=None):
#         super().__init__(root, subdir, hparams, num_events, transform, pre_transform, pre_filter)
#         print(self.__getitem__)

#     # def get_region(self, model):
#     #     return 'region_' + '_'.join([str(i) for i in model['volume_ids']])

#     def get(self, idx):
        
#         event = self.__getitem__(idx)

#         # Process event with pt cuts
#         # if self.hparams["pt_background_cut"] > 0:
#         #     event = background_cut_event(event, self.hparams["pt_background_cut"], self.hparams["pt_signal_cut"])
        
#         # Ensure PID definition is correct
#         # event.y_pid = (event.pid[event.edge_index[0]] == event.pid[event.edge_index[1]]) & event.pid[event.edge_index[0]].bool()
#         # event.pid_signal = torch.isin(event.edge_index, event.signal_true_edges).all(0) & event.y_pid

#         # create new hit map
#         models = config['model_ids']
#         map = torch.zeros_like(event.hid)
#         for model in self.hparams['model_ids']:
#             volume_id = model['volume_ids']
#             homo_ids = event.hid[ torch.isin( event.volume_id, torch.tensor(volume_id) ) ]
#             map[homo_ids] = torch.arange(homo_ids.shape[0])
        

#         data = HeteroData()
#         for model in self.hparams['model_ids']:
#             region = get_region(model)
#             mask = torch.isin( event.volume_id, torch.tensor(model['volume_ids']) )
#             for attr in ['x', 'cell_data', 'pid', 'hid', 'pt', 'primary', 'nhits', 'modules', 'volume_id']:
#                 data[region][attr] = event[attr][mask]
#             data[region]['mask'] = mask
        
#         for model1, model2 in product(self.hparams['model_ids'], self.hparams['model_ids']):
#             # ids = torch.tensor([model1['volume_ids'], model2['volume_ids']])
#             id0, id1 = torch.tensor([model1['volume_ids']]), torch.tensor([model2['volume_ids']])
#             region1, region2 = get_region(model1), get_region(model2)
#             mask0 = torch.isin(event.volume_id[event.edge_index[0]], id0)
#             mask1 = torch.isin(event.volume_id[event.edge_index[1]], id1)
#             mask = mask1 * mask0 #+ torch.isin(event.volume_id[event.edge_index[0]], id2) * torch.isin(event.volume_id[event.edge_index[1]],id1)
#             edge_index = event.edge_index.T[mask].T
#             edge_index = map[edge_index]
#             data[region1, 'connected_to', region2].edge_index = edge_index
#             data[region1, 'connected_to', region2].y = event.y[mask]
#             data[region1, 'connected_to', region2].y_pid = event.y_pid[mask]
#             for truth_edge in ['modulewise_true_edges', 'signal_true_edges']:
#                 mask = torch.isin(event.volume_id[event[truth_edge][0]], id0) * torch.isin(event.volume_id[event[truth_edge][1]], id1) #+ torch.isin(event.volume_id[event[truth_edge][0]], id2) * torch.isin(event.volume_id[event[truth_edge][1]], id1)
#                 data[region1, 'connected_to', region2][truth_edge] = event[truth_edge].T[mask].T
#         return data


In [5]:
files = os.listdir(os.path.join(config['input_dir'], 'train'))
e = torch.load(os.path.join(config['input_dir'], 'train', files[0]))
e

Data(x=[297481, 9], cell_data=[297481, 11], pid=[297481], event_file='/global/cfs/cdirs/m3443/data/ITk-upgrade/processed/full_events_v4/event000015937', hid=[297481], pt=[297481], primary=[297481], nhits=[297481], modules=[297481], modulewise_true_edges=[2, 116389], signal_true_edges=[2, 12588], edge_index=[2, 574596], y=[574596], y_pid=[574596], volume_id=[297481])

In [9]:
from Pipelines.Common_Tracking_Example.LightningModules.GNN.hetero_dataset import LargeHeteroDataset
transform = partial(process_data, pt_background_cut=config['pt_background_cut'], pt_signal_cut=config['pt_signal_cut'], noise=config['noise'], triplets=False, input_cut=None)
dataset = LargeHeteroDataset(root=config['input_dir'], subdir='train', num_events=10, hparams=config, process_function=transform)
data = dataset.get(0)
# homo_data = torch.load(input_dir)
# undirected_data = T.ToUndirected()(data)

In [11]:
from torch_geometric.nn import HeteroConv, HeteroLinear, MLP, GCNConv, MessagePassing, to_hetero
from torch.nn import Module, ModuleDict
from collections import defaultdict

class NodeEncoder(torch.nn.Module):
    def __init__(self, hparams, model) -> None:
        super().__init__()
        self.hparams = hparams

        self.network = make_mlp(
            model['num_features'],
            [hparams["hidden"]] * hparams["nb_node_layer"],
            output_activation=hparams["output_activation"],
            hidden_activation=hparams["hidden_activation"],
            layer_norm=hparams["layernorm"],
            batch_norm=hparams["batchnorm"],
        )

    def forward(self, x):
        return  self.network( x.float() )
    
class HeteroNodeEncoder(torch.nn.Module):
    def __init__(self, hparams) -> None:
        super().__init__()

        self.hparams = hparams

        self.encoders = torch.nn.ModuleDict()
        for model in self.hparams['model_ids']:
            region = get_region(model)
            self.encoders[region] = NodeEncoder(self.hparams, model)
        
    def forward(self, x_dict):
        for model in self.hparams['model_ids']:
            region = get_region(model)
            print(region)
            x_dict[region] = self.encoders[region](x_dict[region][:, : model['num_features']])
        
        return x_dict

In [7]:
encoder = HeteroNodeEncoder(hparams=config)

encoder(data.x_dict)

{'volume_0_1': tensor([[ 0.4559, -0.6419, -0.2931,  ...,  0.0601,  0.2524,  0.7009],
         [ 0.4751, -0.6559, -0.3046,  ...,  0.0529,  0.2268,  0.7113],
         [ 0.4322, -0.6406, -0.2763,  ...,  0.0861,  0.2159,  0.7096],
         ...,
         [-0.9515, -0.9330, -0.9221,  ..., -0.2887,  0.8334, -0.8976],
         [-0.9532, -0.9333, -0.9220,  ..., -0.2904,  0.8344, -0.8974],
         [-0.9538, -0.9331, -0.9231,  ..., -0.2832,  0.8338, -0.8953]],
        grad_fn=<TanhBackward0>),
 'volume_2': tensor([[ 0.5418,  0.9371,  0.8508,  ..., -0.9120,  0.1569,  0.0486],
         [ 0.5762,  0.9358,  0.8506,  ..., -0.9064,  0.1539,  0.0734],
         [ 0.4515,  0.9366,  0.8540,  ..., -0.9223,  0.1309, -0.0043],
         ...,
         [ 0.3760, -0.8509, -0.4800,  ..., -0.7841,  0.7006, -0.4799],
         [ 0.3826, -0.8437, -0.4772,  ..., -0.7881,  0.7012, -0.4746],
         [ 0.4670, -0.7989, -0.5056,  ..., -0.8094,  0.7284, -0.4121]],
        grad_fn=<TanhBackward0>),
 'volume_3': tensor([[ 0

In [12]:
class EdgeEncoder(torch.nn.Module):
    def __init__(self, hparams) -> None:
        super().__init__()
        self.hparams = hparams

        self.network = make_mlp(
            2 * hparams['hidden'],
            [hparams["hidden"]] * hparams["nb_node_layer"],
            output_activation=hparams["output_activation"],
            hidden_activation=hparams["hidden_activation"],
            layer_norm=hparams["layernorm"],
            batch_norm=hparams["batchnorm"],
        )

    def forward(self, x, edge_index):
        src, dst = edge_index
        print("Encoding edges")
        if isinstance(x, tuple):
            x1, x2 = x
            x_in = torch.cat([x1[src], x2[dst]], dim=-1)
        else:
            x_in = torch.cat([x[src], x[dst]], dim=-1)

        return  self.network( x_in )

class HeteroEdgeConv(HeteroConv):

    def __init__(self, convs: dict, aggr: str = "sum"):
        super().__init__(convs, aggr)

    def forward(
        self,
        x_dict: dict,
        edge_index_dict: dict,
        *args_dict,
        **kwargs_dict,
    ) -> dict :

        out_dict = {}
        for edge_type, edge_index in edge_index_dict.items():
            print(edge_type)
            src, rel, dst = edge_type

            str_edge_type = '__'.join(edge_type)
            if str_edge_type not in self.convs:
                continue

            args = []
            for value_dict in args_dict:
                if edge_type in value_dict:
                    args.append(value_dict[edge_type])
                elif src == dst and src in value_dict:
                    args.append(value_dict[src])
                elif src in value_dict or dst in value_dict:
                    args.append(
                        (value_dict.get(src, None), value_dict.get(dst, None)))

            kwargs = {}
            for arg, value_dict in kwargs_dict.items():
                arg = arg[:-5]  # `{*}_dict`
                if edge_type in value_dict:
                    kwargs[arg] = value_dict[edge_type]
                elif src == dst and src in value_dict:
                    kwargs[arg] = value_dict[src]
                elif src in value_dict or dst in value_dict:
                    kwargs[arg] = (value_dict.get(src, None),
                                value_dict.get(dst, None))

            conv = self.convs[str_edge_type]
            print(kwargs)

            if src == dst:
                out = conv(x_dict[src], edge_index, *args, **kwargs)
            else:
                out = conv((x_dict[src], x_dict[dst]), edge_index, *args, **kwargs)

            out_dict[edge_type] = out

        return out_dict

class EdgeClassifier(torch.nn.Module):

    def __init__(self, hparams):
        super().__init__()

        self.network = make_mlp(
            3 * hparams["hidden"],
            [hparams["hidden"]] * hparams["nb_edge_layer"] + [1],
            layer_norm=hparams["layernorm"],
            batch_norm=hparams["batchnorm"],
            output_activation=hparams['output_activation'],
            hidden_activation=hparams["hidden_activation"],
        )

    def forward(self, x, edge_index, edge):
        src, dst = edge_index
        if isinstance(x, tuple):
            x1, x2 = x
            classifier_input = torch.cat([x1[src], x2[dst], edge], dim=-1)
        else:
            classifier_input = torch.cat([x[src], x[dst], edge], dim=-1)
        # classifier_input = torch.cat([x[src], x[dst], edge], dim=-1)
        return self.network(classifier_input)

In [13]:
class InteractionMessagePassing(MessagePassing):
    def __init__(self, hparams, aggr: str = "add", flow: str = "source_to_target", node_dim: int = -2, decomposed_layers: int = 1):
        super().__init__(aggr, flow=flow, node_dim=node_dim, decomposed_layers=decomposed_layers)

        self.hparams=hparams

        # The edge network computes new edge features from connected nodes
        self.edge_encoder = make_mlp(
            2 * (hparams["hidden"]),
            [hparams["hidden"]] * hparams["nb_edge_layer"],
            layer_norm=hparams["layernorm"],
            batch_norm=hparams["batchnorm"],
            output_activation=hparams["output_activation"],
            hidden_activation=hparams["hidden_activation"],
        )

        # The edge network computes new edge features from connected nodes
        self.edge_network = make_mlp(
            3 * hparams["hidden"],
            [hparams["hidden"]] * hparams["nb_edge_layer"],
            layer_norm=hparams["layernorm"],
            batch_norm=hparams["batchnorm"],
            output_activation=hparams["output_activation"],
            hidden_activation=hparams["hidden_activation"],
        )

        # The node network computes new node features
        self.node_network = make_mlp(
            2 * hparams["hidden"],
            [hparams["hidden"]] * hparams["nb_node_layer"],
            layer_norm=hparams["layernorm"],
            batch_norm=hparams["batchnorm"],
            output_activation=hparams["output_activation"],
            hidden_activation=hparams["hidden_activation"],
        )

    def message(self, edge):
        return edge

    def aggregate(self, out, edge_index):

        src, dst = edge_index
        return self.aggr_module(out, dst)[dst.unique()]
    
    def update(self, agg_message, x, edge_index):
        src, dst = edge_index
        indices_to_add = torch.arange(agg_message.shape[0])
        print(dst.unique())
        print(x)
        x[dst.unique()] += agg_message
        
        return x

    def edge_update(self, x, edge, edge_index, *args, **kwargs):
        src, dst = edge_index
        if isinstance(x, tuple):
            x_src, x_dst = x[0][src], x[1][dst]
        else:
            x_src, x_dst = x[src], x[dst]
        out = self.edge_network(torch.cat([x_src, x_dst, edge], dim=-1))
        return out

    def forward(self, x, edge_index, edge):

        if isinstance(x, tuple):
            x_src, x_dst = x
        else:
            x_src, x_dst = x, x

        x_dst = self.propagate(edge_index, x=x_dst, edge=edge)

        return x_dst

class InteractionHeteroConv(HeteroConv):
    def __init__(self, convs: Dict[EdgeType, Module], aggr: Optional[str] = "sum"):
        super().__init__(convs, aggr)

    def edge_forward(self,x_dict: Dict[NodeType, Tensor],
        edge_index_dict: Dict[EdgeType, Adj],
        edge_dict,
        *args_dict,
        **kwargs_dict,
    ) -> Dict[NodeType, Tensor]:

        out_dict = {}
        for edge_type, edge_index in edge_index_dict.items():
            print(edge_type)
            src, rel, dst = edge_type

            str_edge_type = '__'.join(edge_type)
            if str_edge_type not in self.convs:
                continue

            args = []
            for value_dict in args_dict:
                if edge_type in value_dict:
                    args.append(value_dict[edge_type])
                elif src == dst and src in value_dict:
                    args.append(value_dict[src])
                elif src in value_dict or dst in value_dict:
                    args.append(
                        (value_dict.get(src, None), value_dict.get(dst, None)))

            kwargs = {}
            for arg, value_dict in kwargs_dict.items():
                arg = arg[:-5]  # `{*}_dict`
                if edge_type in value_dict:
                    kwargs[arg] = value_dict[edge_type]
                elif src == dst and src in value_dict:
                    kwargs[arg] = value_dict[src]
                elif src in value_dict or dst in value_dict:
                    kwargs[arg] = (value_dict.get(src, None),
                                value_dict.get(dst, None))

            conv = self.convs[str_edge_type]
            edge = edge_dict[edge_type]

            out = conv.edge_update((x_dict[src], x_dict[dst]), edge, edge_index, *args, **kwargs)

            # if src == dst:
            #     out = conv.edge_updater(x_dict[src], edge_index, *args, **kwargs)
            # else:
            #     out = conv.edge_update((x_dict[src], x_dict[dst]), edge_index, *args,
            #             **kwargs)

            out_dict[edge_type] = out

        return out_dict


class HeteroGNN(torch.nn.Module):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        self.node_encoders = HeteroNodeEncoder(self.hparams)
        self.edge_encoders = HeteroEdgeConv({
            (get_region(model0), 'connected_to', get_region(model1)): EdgeEncoder(self.hparams)
            for model0, model1 in product(self.hparams['model_ids'], self.hparams['model_ids'])
        })

        self.convs = torch.nn.ModuleList()

        for _ in range(2):
            conv = InteractionHeteroConv({
                (get_region(model0), 'connected_to', get_region(model1)): InteractionMessagePassing(hparams=self.hparams)
                for model0, model1 in product(self.hparams['model_ids'], self.hparams['model_ids'])
            }, aggr='sum')
            self.convs.append(conv)

        self.edge_classifiers = HeteroEdgeConv({
            (get_region(model0), 'connected_to', get_region(model1)): EdgeClassifier(self.hparams)
            for model0, model1 in combinations_with_replacement(self.hparams['model_ids'], 2)
        })


    def forward(self, x_dict: dict, edge_index_dict: dict):
        x_dict = self.node_encoders(x_dict)  
        edge_dict = self.edge_encoders(x_dict, edge_index_dict) 
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict, edge_dict=edge_dict)
            edge_dict = conv.edge_forward(x_dict, edge_index_dict, edge_dict)

        out = self.edge_classifiers(x_dict, edge_index_dict, edge_dict=edge_dict)

        return out

In [17]:
from torch_geometric.data import DataLoader
import torch.functional as F

class HeteroGNNBase(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        """
        Initialise the Lightning Module that can scan over different GNN training regimes
        """

        # Assign hyperparameters
        self.save_hyperparameters(hparams)
        self.trainset, self.valset, self.testset = None, None, None

    def setup(self, stage):
        # Handle any subset of [train, val, test] data split, assuming that ordering

        if self.trainset is None:
            print("Setting up dataset")
            input_subdirs = [None, None, None]
            input_subdirs[: len(self.hparams["datatype_names"])] = [
                os.path.join(self.hparams["input_dir"], datatype)
                for datatype in self.hparams["datatype_names"]
            ]
            self.trainset, self.valset, self.testset = [
                load_dataset(
                    input_subdir=input_subdir,
                    num_events=self.hparams["datatype_split"][i],
                    **self.hparams
                )
                for i, input_subdir in enumerate(input_subdirs)
            ]

        if (
            (self.trainer)
            and ("logger" in self.trainer.__dict__.keys())
            and ("_experiment" in self.logger.__dict__.keys())
        ):
            self.logger.experiment.define_metric("val_loss", summary="min")
            self.logger.experiment.define_metric("sig_auc", summary="max")
            self.logger.experiment.define_metric("tot_auc", summary="max")
            self.logger.experiment.define_metric("sig_fake_ratio", summary="max")
            self.logger.experiment.define_metric("custom_f1", summary="max")
            self.logger.experiment.log({"sig_auc": 0})
            self.logger.experiment.log({"sig_fake_ratio": 0})
            self.logger.experiment.log({"custom_f1": 0})

    def train_dataloader(self):
        if self.trainset is not None:
            return DataLoader(
                self.trainset, batch_size=1, num_workers=1
            )  # , pin_memory=True, persistent_workers=True)
        else:
            return None

    def val_dataloader(self):
        if self.valset is not None:
            return DataLoader(
                self.valset, batch_size=1, num_workers=1
            )  # , pin_memory=True, persistent_workers=True)
        else:
            return None

    def test_dataloader(self):
        if self.testset is not None:
            return DataLoader(
                self.testset, batch_size=1, num_workers=1
            )  # , pin_memory=True, persistent_workers=True)
        else:
            return None

    def configure_optimizers(self):
        optimizer = [
            torch.optim.AdamW(
                self.parameters(),
                lr=(self.hparams["lr"]),
                betas=(0.9, 0.999),
                eps=1e-08,
                amsgrad=True,
            )
        ]
        scheduler = [
            {
                "scheduler": torch.optim.lr_scheduler.StepLR(
                    optimizer[0],
                    step_size=self.hparams["patience"],
                    gamma=self.hparams["factor"],
                ),
                "interval": "epoch",
                "frequency": 1,
            }
        ]
        return optimizer, scheduler

    def handle_directed(self, batch, edge_sample, truth_sample, sample_indices):

        edge_sample = torch.cat([edge_sample, edge_sample.flip(0)], dim=-1)
        truth_sample = truth_sample.repeat(2)
        sample_indices = sample_indices.repeat(2)

        if ("directed" in self.hparams.keys()) and self.hparams["directed"]:
            direction_mask = batch.x[edge_sample[0], 0] < batch.x[edge_sample[1], 0]
            edge_sample = edge_sample[:, direction_mask]
            truth_sample = truth_sample[direction_mask]

        return edge_sample, truth_sample, sample_indices
    
    def training_step(self, batch, batch_idx):
        
        # truth = batch[self.hparams["truth_key"]]

        # currently, we don't train purity
        # if ("train_purity" in self.hparams.keys()) and (
        #     self.hparams["train_purity"] > 0
        # ):
        #     edge_sample, truth_sample, sample_indices = purity_sample(
        #         truth, batch.edge_index, self.hparams["train_purity"]
        #     )
        # else:
        #     edge_sample, truth_sample, sample_indices = batch.edge_index, truth, torch.arange(batch.edge_index.shape[1])
            
        # edge_sample, truth_sample, sample_indices = self.handle_directed(batch, edge_sample, truth_sample, sample_indices)

        # weight = (
        #     torch.tensor(self.hparams["weight"])
        #     if ("weight" in self.hparams)
        #     else torch.tensor((~truth_sample.bool()).sum() / truth_sample.sum())
        # )

        output_dict = self(batch.x_dict, batch.edge_index_dict)
        truth_dict = batch.truth_dict
        if self.hparams["mask_background"]:
            for key, output in output_dict.items():
                y_subset = batch.truth_dict[key] | ~batch.y_pid_dict[key].bool() # previously the y_pid is filtered by the sample_indices
                output_dict[key], truth_dict[key] = output_dict[key][y_subset], batch.truth_dict[key][y_subset]

        loss = [
            F.binary_cross_entropy_with_logits(
                output, truth.float(), reduction='sum'
            )
            for output, truth in zip(output_dict.values(), truth_dict.values())
        ]
        loss /= np.sum([output.shape[0] for output in output_dict.values()])
        # loss = F.binary_cross_entropy_with_logits(
        #     output, truth_sample.float()#, pos_weight=weight
        # )

        self.log("train_loss", loss, on_step=False, on_epoch=True)

        return loss

    def log_metrics(self, output, sample_indices, batch, loss, log):

        preds = torch.sigmoid(output) > self.hparams["edge_cut"]

        # Positives
        edge_positive = preds.sum().float()

        # Signal true & signal tp
        sig_truth = batch.pid_signal[sample_indices]
        sig_true = sig_truth.sum().float()
        sig_true_positive = (sig_truth.bool() & preds).sum().float()
        sig_auc = roc_auc_score(
            sig_truth.bool().cpu().detach(), torch.sigmoid(output).cpu().detach()
        )

        # Total true & total tp
        tot_truth = (batch.y_pid.bool() | batch.y.bool())[sample_indices]
        tot_true = tot_truth.sum().float()
        tot_true_positive = (tot_truth.bool() & preds).sum().float()
        tot_auc = roc_auc_score(
            tot_truth.bool().cpu().detach(), torch.sigmoid(output).cpu().detach()
        )

        # Eff, pur, auc
        sig_eff = sig_true_positive / sig_true
        sig_pur = sig_true_positive / edge_positive
        tot_eff = tot_true_positive / tot_true
        tot_pur = tot_true_positive / edge_positive

        # Combined metrics
        double_auc = sig_auc * tot_auc
        custom_f1 = 2 * sig_eff * tot_pur / (sig_eff + tot_pur)
        sig_fake_ratio = sig_true_positive / (edge_positive - tot_true_positive)

        if log:
            current_lr = self.optimizers().param_groups[0]["lr"]
            self.log_dict(
                {
                    "val_loss": loss,
                    "current_lr": current_lr,
                    "sig_eff": sig_eff,
                    "sig_pur": sig_pur,
                    "sig_auc": sig_auc,
                    "tot_eff": tot_eff,
                    "tot_pur": tot_pur,
                    "tot_auc": tot_auc,
                    "double_auc": double_auc,
                    "custom_f1": custom_f1,
                    "sig_fake_ratio": sig_fake_ratio,
                },
                sync_dist=True,
            )

        return preds

    def shared_evaluation(self, batch, batch_idx, log=True):

        truth = batch[self.hparams["truth_key"]]
        
        # if ("train_purity" in self.hparams.keys()) and (
        #     self.hparams["train_purity"] > 0
        # ):
        #     edge_sample, truth_sample, sample_indices = purity_sample(
        #         truth, batch.edge_index, self.hparams["train_purity"]
        #     )
        # else:
        #     edge_sample, truth_sample, sample_indices = batch.edge_index, truth, torch.arange(batch.edge_index.shape[1])
            
        # edge_sample, truth_sample, sample_indices = self.handle_directed(batch, edge_sample, truth_sample, sample_indices)

        # weight = (
        #     torch.tensor(self.hparams["weight"])
        #     if ("weight" in self.hparams)
        #     else torch.tensor((~truth_sample.bool()).sum() / truth_sample.sum())
        # )
        
        # output = self(batch.x.float(), edge_sample, batch.volume_id).squeeze()

        output_dict = self(batch.x_dict, batch.edge_index_dict)
        truth_dict = batch.truth_dict
        if self.hparams["mask_background"]:
            for key, output in output_dict.items():
                y_subset = batch.truth_dict[key] | ~batch.y_pid_dict[key].bool() # previously the y_pid is filtered by the sample_indices
                output_dict[key], truth_dict[key] = output_dict[key][y_subset], truth_dict[key][y_subset]

        loss = [
            F.binary_cross_entropy_with_logits(
                output, truth.float(), reduction='sum'
            )
            for output, truth in zip(output_dict.values(), truth_dict.values())
        ]
        loss /= np.sum([output.shape[0] for output in output_dict.values()])

        # if self.hparams["mask_background"]:
        #     y_subset = truth_sample | ~batch.y_pid[sample_indices].bool()
        #     subset_output, subset_truth_sample = output[y_subset], truth_sample[y_subset]
        #     loss = F.binary_cross_entropy_with_logits(
        #         subset_output, subset_truth_sample.float().squeeze(), pos_weight=weight
        #     )            
        # else:
        #     loss = F.binary_cross_entropy_with_logits(
        #         output, truth_sample.float().squeeze(), pos_weight=weight
        #     )

        # try:
        #     preds = self.log_metrics(output, sample_indices, batch, loss, log)
        #     return {"loss": loss, "preds": preds, "score": torch.sigmoid(output)}
        # except:
        return {"loss": loss, 
                # "score": torch.sigmoid(output)
            }

    def validation_step(self, batch, batch_idx):

        outputs = self.shared_evaluation(batch, batch_idx)

        return outputs["loss"]

    def test_step(self, batch, batch_idx):

        outputs = self.shared_evaluation(batch, batch_idx, log=False)

        return outputs

    def test_step_end(self, output_results):

        print("Step:", output_results)

    def test_epoch_end(self, outputs):

        print("Epoch:", outputs)

    def optimizer_step(
        self,
        epoch,
        batch_idx,
        optimizer,
        optimizer_idx,
        optimizer_closure=None,
        on_tpu=False,
        using_native_amp=False,
        using_lbfgs=False,
    ):
        # warm up lr
        if (self.hparams["warmup"] is not None) and (
            self.current_epoch < self.hparams["warmup"]
        ):
            lr_scale = min(
                1.0, float(self.current_epoch + 1) / self.hparams["warmup"]
            )
            for pg in optimizer.param_groups:
                pg["lr"] = lr_scale * self.hparams["lr"]

        # update params
        optimizer.step(closure=optimizer_closure)
        optimizer.zero_grad()

        
class LargeGNNBase(HeteroGNNBase):
    def __init__(self, hparams):
        super().__init__(hparams)

    def setup(self, stage, process_function):
        # Handle any subset of [train, val, test] data split, assuming that ordering
        splits = np.array(self.hparams['datatype_split']) 
        splits = (self.hparams['n_events'] * splits / np.sum(splits)).astype(np.int32)

        self.trainset, self.valset, self.testset = [
            LargeHeteroDataset(
                self.hparams['input_dir'],
                subdir,
                self.hparams,
                split,
                process_function
            )
            for subdir, split in zip(self.hparams['datatype_names'], self.hparams['datatype_split'])
        ]

        if (
            (self.trainer)
            and ("logger" in self.trainer.__dict__.keys())
            and ("_experiment" in self.logger.__dict__.keys())
        ):
            self.logger.experiment.define_metric("val_loss", summary="min")
            self.logger.experiment.define_metric("sig_auc", summary="max")
            self.logger.experiment.define_metric("tot_auc", summary="max")
            self.logger.experiment.define_metric("sig_fake_ratio", summary="max")
            self.logger.experiment.define_metric("custom_f1", summary="max")
            self.logger.experiment.log({"sig_auc": 0})
            self.logger.experiment.log({"sig_fake_ratio": 0})
            self.logger.experiment.log({"custom_f1": 0})

In [2]:
from Pipelines.Common_Tracking_Example.LightningModules.GNN.hetero_gnn_base import PyGHeteroGNNBase

gnn = PyGHeteroGNNBase(config)

TypeError: __init__() missing 1 required positional argument: 'hparams'